In [ ]:
from pycalphad import Database, equilibrium, variables as v, calculate
db_name = 'models/sgsol_2021_pycalphad.tdb'
db = Database(db_name)

In [ ]:
# constants:
P = 101325

# Компоненты и фазы

In [ ]:
import random

random.seed(10)
all_components = list(db.elements)
all_phases =  list(db.phases)
all_phases.sort()
all_components.sort()
print('Количество фаз =' + str(len(all_phases)))
print('Количество компонент =' + str(len(all_components)))
print(all_components)
# components = random.sample(all_components, 8)
components = ['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE']

print(components)


# Бинарные фазы для компонент

In [ ]:
counter = 0 
components_phases = {}
for comp in components:
    components_phases[comp] = set()
for item in db.phases.items():
    # item[0] - name, item[1] - content
    # item[1] - name, constituents, sublattices, model_hints
    if len(item[1].constituents)>4:
        counter += 1
    
    for species in item[1].constituents[0]:
        # name, constituents, charge
        for phase in components_phases.keys():
            if phase in species.name:
                components_phases[phase].add(item[0])

print(counter)
print(components_phases)

for comp, phases in components_phases.items():
    mask = [1 if phase in phases else 0 for phase in all_phases]
    components_phases[comp] = int(''.join(map(str, mask)), 2)

In [ ]:
for comp, phases in components_phases.items():
    print(f'{comp}: {bin(phases)}')

In [ ]:
def getPhases(int_mask):
    return [all_phases[i] for i in range(len(all_phases)) if (int_mask & (1 << i)) > 0]
def getAllPhases(comp_phases):
    int_mask = 0
    for phases in comp_phases.values():
        int_mask |= phases
    return getPhases(int_mask)
# print(getAllPhases(components_phases))
print(getPhases(components_phases['SE']))

In [ ]:
for comp, phases in components_phases.items():
    print(comp)
    print(getPhases(phases))
    for phase in getPhases(phases):
        if str(phase) in db.symbols:
            print(f'{phase} in {db.symbols}')

# Функции

In [ ]:
import re

# read the text file
text = ""
function_start = False
with open(db_name, 'r') as f:
    for line in f:
        if "Standard functions for the elements" in line:
            function_start = True
        if "Data for the elements from unary database" in line:
            function_start = False

        if function_start:
            text += line

component_functions = {}
# find all the component names in the text
component_names = re.findall(r'\$ ([A-Z][a-z]*)', text)
for component in component_names:
    # find the text between the component name and the next component name
    pattern = r'\$ {}.*?\$ [A-Z][a-z]*'.format(component)
    component_text = re.search(pattern, text, re.DOTALL).group()
    
    # find all the function names in the component text
    function_names = re.findall(r'FUNCTION ([A-Z]*)', component_text)
    
    component_functions[component] = function_names
component_functions = {k.upper():v for k,v in component_functions.items()}

In [ ]:
# print the functions for each component
for component, functions in component_functions.items():
    if len(functions) > 0:
        print(f"{component} : {functions}")

In [ ]:
functions = set()
for component in components:
    if str(component) in component_functions.keys():
        functions.update(component_functions[component])
    else:
        print(f'Check - {component}')
print(functions)

In [ ]:
from symengine import Piecewise, Symbol, lib
import math

temp = 1200
T = Symbol('T')
print(db.symbols['GBCCGD'])
print(type(temp))
expr = db.symbols['GBCCGD'].subs(T, temp)
print(expr)
# print(db.symbols['GBCCGD'].subs(T, 800).simplify())
# print (math.log(14))


In [ ]:
from collections import OrderedDict

function_value = OrderedDict()
for func in functions:
    expression = Piecewise()
    print(func)
    if func not in db.symbols:
        print('Wrong function: ' + str(func))
        function_value[func] = None
        continue
    
    for arg in db.symbols[func].args:
        # condition = arg[0].subs(T, temp)
        # expression = arg[1].subs(T, temp)
        # result.append([condition, expression])
        if arg.is_Add:
            expression = arg.subs(T, temp).evalf()
        elif arg.is_Float or arg.is_Integer:
            expression = arg
        else:
            bool_res = False
            if len(arg.atoms()) > 0:
                bool_res = arg.subs(T, temp)
            if arg.is_Boolean:
                bool_res = arg
            if bool_res: 
                function_value[func] = expression
                break

print(function_value)

# Расчет фазовых равновесий

In [31]:
necessary_component = 'VA'
temp = 1200
if necessary_component not in components:
    components.append(necessary_component)

conditions = {v.T:temp,v.P:P}
# TODO: make random amount of component
amount = 1.0/len(components)
for comp in components:
    if comp != necessary_component:
        conditions[v.X(comp)] = amount
eq = equilibrium(db,components,getAllPhases(components_phases),conditions, verbose=True)
gibbs_energy = eq.GM

Components: AL AL1 AL1AU AL1AU2 AL1AU4 AL2 AL2AU AL2AU5 AU HG RH SE SM V VA ZR
Phases: 

DofError: MOB4: Sublattices of (frozenset({Species('VA', 'VA1'), Species('TI', 'TI1'), Species('MO', 'MO1')}), frozenset({Species('VA', 'VA1'), Species('B', 'B1')})) contains only VA (VACUUM) constituents

In [32]:
gibbs_energy = '???'

# Текущие результаты:

In [33]:
print('Список компонентов:')
print(components)

Список компонентов:
['V', 'AU', 'RH', 'SM', 'ZR', 'AL', 'HG', 'SE', 'VA']


In [34]:
print('Все фазы для компонентов из списка:')
print(getAllPhases(components_phases))

Все фазы для компонентов из списка:
['AG2BA', 'AG2BA3', 'AG2CA', 'AG2DY', 'AG2ER', 'AG3BE8', 'AGCA3', 'AGCD', 'AGCD_ETA', 'AGCE', 'AGDY', 'AGER', 'AGGD_B2', 'AGIN2', 'AGLA_B2', 'AGMG3', 'AGND', 'AGSB_ORTHO', 'AGSC', 'AGSR', 'AGTI2', 'AL11SM3_HT', 'AL11SM3_LT', 'AL11_CEND3_H', 'AL28IR9', 'AL2AU', 'AL3CO', 'AL3DY_D024', 'AL3GD', 'AL3HF2', 'AL3HF4', 'AL3HF_D022', 'AL3HF_D023', 'AL3IR_D018', 'AL3LA', 'AL3M_D022', 'AL3NB', 'AL3NI', 'AL3NI2', 'AL3NI5', 'AL3PD', 'AL3PD2_D513', 'AL3PD5', 'AL3PR', 'AL3PT2', 'AL3PT5', 'AL3PU_L12', 'AL3U_L12', 'AL5W', 'AL63MO37', 'AL9CO2', 'AL9IR2', 'ALCU_THETA', 'ALFESI_GAMMA', 'ALMGZN_TAU', 'ALPHA_GE3RU2', 'ALPR', 'ALSM2', 'ASGE', 'ASNI', 'AU36ND17', 'AU4PR3', 'AU6LA', 'AUGA2_C1', 'AUPR_GAMMA', 'BA5PB3_D8L', 'BA6MG23_D8A', 'BABI3', 'BCC_B2', 'BCT_A5', 'BE12MO_D2B', 'BE12V_D2B', 'BI2LA', 'BI3ER5', 'BILA2', 'BILA_B1', 'BILI3_D03', 'BILI_HT', 'BIMN_B82', 'BIND_B1', 'BIRB3_D03', 'BIU_B1', 'CA5PB3', 'CA5SI3', 'CA5SN3_D8L', 'CD2PU', 'CD2SR', 'CD3SR5_D8L', 'CD6Y', 'CD

In [35]:
int_mask = 0
for phases in components_phases.values():
    int_mask |= phases
print(f'Все фазы для компонентов из списка(бинарные): {bin(int_mask)}')

Все фазы для компонентов из списка(бинарные): 0b1000000000000000000000000000000000000000000000000000010000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111011111111111111111111111111100000111111111100101111111111111111111111111111100000000000000000000000000111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111110100000010000000000000000000000001110000000000000000000000001100000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000010000000000000000000000000000100100000000100000000000000100000001000000000010000000001000001000000000000010010000000000000000000000000000000000000000000000000000000000000111000000000000000000000010000000000000010000000010000000000000000100001000000000000000

In [36]:
print('Параметры расчета:')
print(f'Давление {P}, Температура {temp}, Количестов вещества по {amount} для компоненты')

Параметры расчета:
Давление 101325, Температура 1200, Количестов вещества по 0.1111111111111111 для компоненты


In [37]:
print('Значения функций для компонентов:')
print(function_value)

Значения функций для компонентов:
OrderedDict([('GLIQAL', -57996.2850123662), ('GHSERHG', -112210.200050902), ('GLIQRH', -48315.478673478), ('GHSERV', -57463.158455921), ('GLIQGD', -102198.298584263), ('GFCCHG', 10046.6 + GHSERHG), ('GFE', None), ('GBCCGD', -104573.24219628), ('GHCPRH', 2400.0 + GHSERRH), ('GHCPAL', 3321.0 + GHSERAL), ('GHSERRH', 0), ('GFENI', None), ('GBCCZR', -69104.2656226965), ('GFCCGD', 500.0 + GHSERGD), ('GBCCCZ', -5725.0), ('GLIQV', 9415.59444185221 + GHSERV), ('GLIQSE', 0), ('GLAVAL', 5000.0 + GHSERAL), ('U', None), ('GBCCAU', 2930.0 + GHSERAU), ('GHSERVZ', -57463.158455921), ('GHSERGD', -105270.77857148), ('GHSERAL', -54976.3276402473), ('GBCCAL', 4307.4 + GHSERAL), ('GHSERSE', 0), ('GHSERZR', -68894.6050887857), ('GBCCL', None), ('LB', None), ('GHSERAU', -77272.6856248379), ('GLIQAU', 1288.9608 + GHSERAU), ('GBCCRH', 13360.0 + GHSERRH), ('GFCCZR', 6520.0 + GHSERZR), ('GB', None), ('GHCPAU', 2160.75 + GHSERAU), ('GBCCMZ', 0.0), ('GHCPHG', -103109.498678239), (

In [38]:
print('Значение энергии Гиббса для заданых параметров:')
print(gibbs_energy)

Значение энергии Гиббса для заданых параметров:
???
